In [11]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [12]:
api_key = input("Please enter your YouTube Data API key: ")
channel_id='UCtAXJ4DNpshfVjLIMq9pIRw'
youtube= build('youtube','v3',developerKey=api_key)

In [13]:
def get_channel(youtube, channel_id):
    all_data = []
    request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=channel_id
    )
    response = request.execute() 
    
    # Check if 'items' key exists in the response
    if 'items' in response:
        for i in range(len(response['items'])):
            data = dict(
                channelName=response['items'][i]['snippet']['title'],
                subscribers=response['items'][i]['statistics']['subscriberCount'],
                views=response['items'][i]['statistics']['viewCount'],
                totalVideos=response['items'][i]['statistics']['videoCount'],
                playlistId=response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
            )
            all_data.append(data)
    else:
        print("No 'items' found in response.")
    
    return pd.DataFrame(all_data)



In [14]:
channel_stats=get_channel(youtube,channel_id)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Rohit & Kanupriya,2290000,2181237212,329,UUtAXJ4DNpshfVjLIMq9pIRw


In [15]:
playlistid=channel_stats['playlistId']
playlistid

0    UUtAXJ4DNpshfVjLIMq9pIRw
Name: playlistId, dtype: object

In [16]:
def get_video_ids(youtube, playlist_id,video_ids):

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [17]:
video_ids = []
for i in range(len(playlistid)):
    playlist_id=playlistid[i]
    Video_ids=get_video_ids(youtube, playlist_id,video_ids)
    
len(Video_ids)

329

In [18]:
def get_comments_in_videos(youtube, video_ids):
    all_comments = []

    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
            )
            response = request.execute()
        
            for comment in response['items']:
                data = {
                    'videoId': video_id,
                    'commenter_name': comment['snippet']['topLevelComment']['snippet']['authorDisplayName'],        
                    'comments': comment['snippet']['topLevelComment']['snippet']['textOriginal'],
                    'commentProfileImageUrl': comment['snippet']['topLevelComment']['snippet']['authorProfileImageUrl']
                }
                all_comments.append(data)
                
                # Add replies
               

        except Exception as e: 
            # When an error occurs - most likely because comments are disabled on a video
            print(f'Could not get comments for video {video_id}: {e}')
        
    return pd.DataFrame(all_comments)


In [19]:
comments = get_comments_in_videos(youtube, video_ids)
comments

,videoId,commenter_name,comments,commentProfileImageUrl
0,JXvx90MmuG8,@RohitandKanu,Do let me know if you guys would like me to go...,https://yt3.ggpht.com/ciKJQ6_fiiQU3caIT8GWU1-5...
1,JXvx90MmuG8,@anshuman5331,Drinking Cold water is the cure.\nBath 2 times...,https://yt3.ggpht.com/ytc/AIdro_mFHc65ZAwKKP2r...
2,JXvx90MmuG8,@sathwiktalents,Bro\nI've been working on music and video edit...,https://yt3.ggpht.com/Kl5aW5w9Q2QhGkflaVdpZpcS...
3,JXvx90MmuG8,@sharma-rx7vg,Wow❤❤❤take care of yourself because we need u ...,https://yt3.ggpht.com/ytc/AIdro_lu2GvO15oLoFqt...
4,JXvx90MmuG8,@abhishekhawk2762,@28:00 Real men don't need manual❌ Wise men do...,https://yt3.ggpht.com/ytc/AIdro_mSWXlwJm2HPlDb...
...,...,...,...,...
6416,xxwNglca0Bo,@dmcindianfan5374,Just went back 10 years ago 😂😅,https://yt3.ggpht.com/Q8dsv0uzwkxGyb8JG9uUoF5L...
6417,xxwNglca0Bo,@coolbreeze1262,Is the record still to her ? ❤,https://yt3.ggpht.com/ytc/AIdro_k2MBvG9XLwRN9I...
6418,xxwNglca0Bo,@Notebook-ur1st,Wow!! She looks different 😳 \nI follow them fo...,https://yt3.ggpht.com/oKSbB0Ayy5knZP0XLiLEStDe...
6419,xxwNglca0Bo,@Prathikraj_mysorean,one can see Rohit's Journey 😂,https://yt3.ggpht.com/D7jURbUnRQ-rWwSYl8YLWMPL...


In [20]:
comments.to_csv('Data\\commentsData.csv', index=False)